# Wide-and-Deep ML: Model Preparation

In this notebook, we train and evaluate the wide-and-deep collaborative filtering recommender using features engineered in the prior notebook.

## 1. Prepare the data

In [1]:
# import modules

import pandas as pd

In [2]:
df1 = pd.read_csv('../data/user_movie_interaction.csv')

In [5]:
df1.shape

(71554, 8)